In [1]:
using Symata

INFO: Recompiling stale cache file /home/lapeyre/.julia/lib/v0.6/Symata.ji for module Symata.


# Interface with Julia

This notebook gives examples of Symata interacting with its host language Julia.

## Writing and using Julia functions

It is very easy to write elegant, fast, powerful, flexible, Julia code from within Symata.

As a first example, we multiply powers of elements in two arrays and sum the results. We first try two different methods using pure Symata. Then we write a Julia function to do the same thing.

Here are the arrays.

In [2]:
x1 = Range(10.0^3)
y1 = Range(10.0^3);

Define a procedural function in Symata to compute the sum.

In [3]:
g(x_, y_) := Module([s=0],
        begin
          For(i=1, i<=Length(x), i += 1, s += x[i]^2 * y[i]^(-3))
          s
        end)

Apply this function and time the result.

In [4]:
g(x1,y1)

resultS1 = Timing(g(x1,y1))

L"$$  \left[ 0.080621321,7.485470860550343 \right]  $$"

In general, it is faster to use mapping and functional methods in Symata.
We can compute the sum like this.

In [5]:
Apply(Plus, x1^2 / y1^3)

resultS2 = Timing(Apply(Plus, x1^2 / y1^3))

L"$$  \left[ 0.060660492,7.485470860550343 \right]  $$"

The second method is indeed a bit faster.

In [6]:
resultS1[1]/resultS2[1]

L"$$ 1.3290581454565187 $$"

### Writing a Julia function while in Symata mode

Define a Julia function to do the sum. We choose a functional method.

In [7]:
jfunc = J( (x,y) -> sum(u -> u[1]^2 / u[2]^(3), zip(x,y)) );

In [8]:
jfunc(x1,y1)
Timing(jfunc(x1,y1))

resultJ = Timing(jfunc(x1,y1))

L"$$  \left[ 0.000288341,7.485470860550343 \right]  $$"

The Julia function is much faster.

In [9]:
[resultS1[1], resultS2[1] ] / resultJ[1]

L"$$  \left[ 279.6040833596332,210.37761539288553 \right]  $$"

We will explain later why it is possible to write such simple and fast code that operates on Symata expressions.

The function `f` also works on symbolic expressions.

In [10]:
jfunc([a+b, c+d],[u+v,y+z])

L"$$ \frac{ \left( a + b \right) ^{2}}{ \left( u + v \right) ^{3}} + \frac{ \left( c + d \right) ^{2}}{ \left( y + z \right) ^{3}} $$"

### Evaluating Symata expressions from Julia

The macro `@sym` evaluates Symata code while in Julia.

First, we switch to Julia mode.

In [11]:
Julia();

Create a Symata expression and bind it to the Julia variable `expr`.

In [12]:
expr = @sym a + b

:a + :b

There are julia functions corresponding to many Symata expression heads.

In [13]:
Expand(expr^2)

:a^2 + :b^2 + 2:a*:b

Return to Symata mode

In [14]:
isymata()

### Calling an existing Julia function

To define the Julia function, we used the Symata function `J()`. The arguments of `J` are interpreted as pure Julia code, with no translation. It is as if we temporarily enter Julia mode. In fact, we could have defined the function in Julia. Let's try that.

First, we enter Julia mode.

In [15]:
Julia();

Everything we type will be interpreted as Julia language expressions. We write the Julia function. We will explain later how the function works.

In [16]:
fj(x,y) = sum(u -> u[1]^2 / u[2]^(3), zip(x,y));

Return to Symata mode.

In [17]:
isymata();

We set the Symata variable `fj` to the Julia function `fj`. The Julia function was written in the `Main` module. (NB we may change this so Julia functions are evaluated in the Symata module)

In [18]:
fj = J( Main.fj );

fj(x1,y1)

L"$$ 7.485470860550343 $$"

We can use `J()` in this way to call any existing Julia function...

    time()

      Get the system time in seconds since the epoch, with fairly high (typically,   
      microsecond) resolution.


In [19]:
J(time)()

L"$$ 1.479679060153872e9 $$"

## Julia functions for Symata

How does the Julia function 
```
jfunc = (x,y) -> sum(u -> u[1]^2 / u[2]^(3), zip(x,y))
```
work ?

As in Symata, `x -> body` defines a pure, or anonymous, function. `zip` returns a list of pairs of elements from two lists. In fact, it returns a virtual list, called an *iterator*, which is more efficient. These pairs are supplied sequentially to the to the function, and the results are summed. No intermediate arrays are formed.

Symata expressions are *iterable objects* in Julia. Most Julia code that operates on iterable objects will work with Symata expressions. `zip` takes two iterable objects and returns an iterable object.
In Julia, Symata expressions are of type `Mxpr`. Notice that we did not write `Mxpr` anywhere in the code. The first  time `jfunc` is called with Symata expressions, Julia compiles a method to handle just this case. The compiler is typically very good at writing code optimized for the input type.

All of this means that the author of Symata wrote no code to implement `zip` or `sum` for Symata expressions.

In fact `jfunc` can be called with many types objects. To demonstrate this, we perform the sum operation on a Symata list and a Julia Array.

We set the Symata variable `y2` to a Julia `Array` of 1000 numbers.

In [20]:
y2 = J(linspace(1,1000.0,1000))

L"$$ linspace(1.0,1000.0,1000) $$"

Notice that 1000 numbers were not printed. `linspace` returns a virtual array, that is an iterator. We call `jfunc` twice. The first time, Julia compiles a method for the input types which takes some (not much) time.

In [21]:
jfunc(x1,y2)
resultJ2 = Timing(jfunc(x1,y2))

L"$$  \left[ 0.000290884,7.485470860550343 \right]  $$"

Notice that summing over the two types of arrays is a bit slower in this case than using two Symata arrays.

In [22]:
resultJ[1]/resultJ2[1]

L"$$ 0.9912576834751998 $$"

Now we call `jfunc` on two Julia abstract arrays.

In [23]:
jfunc(y2,y2)
resultJ3 = Timing(jfunc(y2,y2))

L"$$  \left[ 6.0417e-5,7.485470860550343 \right]  $$"

Operating on these Julia arrays is about 10 times faster in this case than including a Symata array.

In [24]:
resultJ2[1]/resultJ3[1]

L"$$ 4.814605160799113 $$"

### `Unpack`:   Importing Julia arrays into Symata

Recall the abstract Julia array that we created above

In [25]:
y2

L"$$ linspace(1.0,1000.0,1000) $$"

We import `y2` into Symata using `Unpack`. (`Unpack` currently works only with one dimensional arrays.)

In [26]:
y3 = Unpack(y2);

The result is a Symata list. We check that it is indeed a list of the expected length and the first and last elements.

In [27]:
[Head(y3), Length(y3), y3[1], y3[-1]]

L"$$  \left[ \text{List},1000,1.0,1000.0 \right]  $$"

`y3` is equal to `y1`, which was created with Range.

In [28]:
y3 == y1 == Range(10.0^3)

L"$$ \text{True} $$"

`y3` is not a `packed` array, but an ordinary Symata array, a list.

In [29]:
y3[1] = "cat"

L"$$ \text{\"cat\"} $$"

In [30]:
y3[1:10]

L"$$  \left[ \text{\"cat\"},2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0 \right]  $$"

Of course, we can unpack not just abstract Julia arrays, but physical arrays as well. The Julia function for converting an abstract array to a physical array is `collect`.

In [31]:
y4 = J(collect)(y2);

Notice what we did here. `J(collect)` gets a Symata reference to the Julia function `collect`. We then call the imported function on the Symata variable `y2`, which refers to an abstract Julia array.

Now we have a physical Julia array

In [32]:
[Head(y4), y4[1]]

L"$$  \left[ \text{Array{Float64,1}},1.0 \right]  $$"

You can also construct a Symata expression directly in Julia, like this.

In [33]:
y5 = J(mxpr(:List, Any[collect(linspace(1.0,1000,1000))...]));

In [34]:
[Head(y5), y5[-1]]

L"$$  \left[ \text{List},1000.0 \right]  $$"

### `Pack`:   Converting Symata arrays to Julia arrays.


Recall that `y1` and `y3` are both Symata lists.

In [35]:
[Head(y1), Head(y3)]

L"$$  \left[ \text{List},\text{List} \right]  $$"

We unpack these lists to Julia arrays like this.

In [36]:
y6 = Pack(y1)
y7 = Pack(y3);

What type of object was created ?

In [37]:
[Head(y6), Head(y7)]

L"$$  \left[ \text{Array{Float64,1}},\text{Array{Any,1}} \right]  $$"

The first array is of type `Float64` and the second of type `Any`. They are different because, while all elements of `y1` are floating point numbers, we set the first element of `y7` to `"cat"`. When copying arrays, Julia creates a container of the most specific type that will contain all elements.